In [1]:
# takes 11m for 5,000,000 pair summaries.
# create pair_summaries for training

# Next steps:
# acbl_board_results_cleanup.ipynb cleans acbl_board_results_raw file.

# Previous steps:
# download-acbl-details-make-json.ipynb

# todo:
# filter out non-pair events
# predict percentage for each board result
# predict rank
# combine with board results to predict tricks taken

In [2]:
import config
import pandas as pd
import pathlib
import pickle
import sqlalchemy
import mlBridgeLib

In [3]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [4]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')
bprPath = rootPath.joinpath('bpr')

In [5]:
# using pathlib to create sqlite path.
db_connection_string = 'sqlite:///'+acblPath.joinpath('acbl-details.sqlite').as_posix()
db_connection_string

'sqlite:///e:/bridge/data/acbl/acbl-details.sqlite'

In [6]:
def db_to_frames_dict(engine):
    meta = sqlalchemy.MetaData()
    meta.reflect(engine)
    tables = meta.tables.keys()
    cnx = engine.raw_connection()
    return {t: pd.read_sql(f'SELECT * FROM {t}', cnx ) for t in tables}

In [7]:
# create dict of sql SELECT statements.
sql_selects_d = {
    # WHERE is_eligible = 1 ?
    'pair_summaries':'SELECT id AS pair_summary_id, section_id, pair_number, direction, strat, percentage, is_eligible FROM pair_summaries',
    'players':'SELECT id AS player_id, pair_summary_id, id_number AS player_number, name AS player_name, mp_total FROM players',
    'awards':'SELECT player_id, id_number as player_number, type, rank FROM awards',
    'sections':'SELECT id AS section_id, session_id, name AS section_name FROM sections',
    'sessions':'SELECT id AS session_id, event_id, hand_record_id, game_date FROM sessions',
    'events':'SELECT id AS event_id, name AS event_name, club_name, club_id_number, type as event_type, rating, board_scoring_method, stratification_type, winner_type, club_class FROM events',
    'strats':'SELECT id AS strat_id, event_id, number AS strats_number, label AS strats_label, [limit] AS strats_limit FROM strats',
    'strat_place':'SELECT id AS strat_place_id, pair_summary_id, strat_number AS strat_place_number, rank AS strat_place_rank, type AS strat_place_type FROM strat_place',
    'stratValues':'SELECT id AS stratValues_id, events AS event_id, label AS stratValues_label, [limit] AS stratValues_limit FROM stratValues'
    }

In [8]:
# takes 2m30s
dfs = {}
for k,v in sql_selects_d.items():
    engine = sqlalchemy.create_engine(db_connection_string) #, echo=True)
    with engine.connect() as con:
        if False:
            dfs = db_to_frames_dict(engine)
        else:
            print(f'{k}:{v}')
            dfs[k] = pd.read_sql(v, con)
    engine.dispose()

pair_summaries:SELECT id AS pair_summary_id, section_id, pair_number, direction, strat, percentage, is_eligible FROM pair_summaries
players:SELECT id AS player_id, pair_summary_id, id_number AS player_number, name AS player_name, mp_total FROM players
awards:SELECT player_id, id_number as player_number, type, rank FROM awards
sections:SELECT id AS section_id, session_id, name AS section_name FROM sections
sessions:SELECT id AS session_id, event_id, hand_record_id, game_date FROM sessions
events:SELECT id AS event_id, name AS event_name, club_name, club_id_number, type as event_type, rating, board_scoring_method, stratification_type, winner_type, club_class FROM events
strats:SELECT id AS strat_id, event_id, number AS strats_number, label AS strats_label, [limit] AS strats_limit FROM strats
strat_place:SELECT id AS strat_place_id, pair_summary_id, strat_number AS strat_place_number, rank AS strat_place_rank, type AS strat_place_type FROM strat_place
stratValues:SELECT id AS stratValues_

In [9]:
for k,v in dfs.items():
    display(k,v.head(),v.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5440994 entries, 0 to 5440993
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   pair_summary_id  int64  
 1   section_id       int64  
 2   pair_number      int64  
 3   direction        object 
 4   strat            float64
 5   percentage       float64
 6   is_eligible      float64
dtypes: float64(3), int64(3), object(1)
memory usage: 290.6+ MB


'pair_summaries'

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible
0,1848542,119248,4,None,1.00,62.50,1.00
1,1848539,119248,1,None,1.00,57.29,1.00
2,1848536,119248,3,None,1.00,51.04,1.00
3,1848540,119248,6,None,1.00,51.04,1.00
4,1848537,119248,2,None,1.00,44.79,1.00


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10880585 entries, 0 to 10880584
Data columns (total 5 columns):
 #   Column           Dtype  
---  ------           -----  
 0   player_id        int64  
 1   pair_summary_id  int64  
 2   player_number    object 
 3   player_name      object 
 4   mp_total         float64
dtypes: float64(1), int64(2), object(2)
memory usage: 415.1+ MB


...

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874295 entries, 0 to 874294
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   stratValues_id     874295 non-null  object
 1   event_id           874295 non-null  int64 
 2   stratValues_label  874295 non-null  object
 3   stratValues_limit  874295 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 26.7+ MB


'stratValues'

,stratValues_id,event_id,stratValues_label,stratValues_limit
0,100961-1,100961,A,0
1,110886-1,110886,A,0
2,114736-1,114736,A,0
3,126385-1,126385,A,0
4,126385-2,126385,B,100


None

In [10]:
pair_summaries_dtypes = {
    'pair_summary_id':'int64',
    'section_id':'int64',
    'pair_number':'int64',
    'direction':'string',
    'strat':'Int8',
    'percentage':'float32',
    'is_eligible':'boolean'
}
df_pair_summaries = dfs['pair_summaries']
df_pair_summaries = df_pair_summaries.astype(pair_summaries_dtypes)
display(df_pair_summaries.isna().sum(), df_pair_summaries.info(), df_pair_summaries.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5440994 entries, 0 to 5440993
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   pair_summary_id  int64  
 1   section_id       int64  
 2   pair_number      int64  
 3   direction        string 
 4   strat            Int8   
 5   percentage       float32
 6   is_eligible      boolean
dtypes: Int8(1), boolean(1), float32(1), int64(3), string(1)
memory usage: 207.6 MB


pair_summary_id         0
section_id              0
pair_number             0
direction          477034
strat                  32
percentage           7999
is_eligible        878739
dtype: int64

None

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible
0,1848542,119248,4,<NA>,1,62.50,True
1,1848539,119248,1,<NA>,1,57.29,True
2,1848536,119248,3,<NA>,1,51.04,True
3,1848540,119248,6,<NA>,1,51.04,True
4,1848537,119248,2,<NA>,1,44.79,True


In [11]:
players_dtypes = {
    'player_id':'int64',
    'pair_summary_id':'int64',
    'player_number':'string',
    'player_name':'string',
    'mp_total':'float32'
}
df_players = dfs['players']
df_players = df_players.astype(players_dtypes)
display(df_players.isna().sum(), df_players.info(), df_players.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10880585 entries, 0 to 10880584
Data columns (total 5 columns):
 #   Column           Dtype  
---  ------           -----  
 0   player_id        int64  
 1   pair_summary_id  int64  
 2   player_number    string 
 3   player_name      string 
 4   mp_total         float32
dtypes: float32(1), int64(2), string(2)
memory usage: 373.6 MB


player_id               0
pair_summary_id         0
player_number           0
player_name             0
mp_total           305921
dtype: int64

None

,player_id,pair_summary_id,player_number,player_name,mp_total
0,3695653,1848542,8052557,"Robson, Dave",693.89
1,3695652,1848542,tmp:1de5c0e8-a5a2-4433-a232-caa798c84939,"Gamache, Len(swap names)",19.00
2,3695646,1848539,tmp:1238a3ab-2652-4f75-921b-767aae646b2a,"Steele, Bruce",16.00
3,3695647,1848539,tmp:9f01395a-c179-4283-adf3-886ff6aed8aa,"Hlady, Henny",NaN
4,3695641,1848536,5637562,"Williams, Peter",139.21


In [12]:
awards_dtypes = {
    'player_id':'int64',
    'player_number':'string',
    'type':'string',
    'rank':'string'
}
df_awards = dfs['awards']
df_awards = df_awards.astype(awards_dtypes)
display(df_awards.isna().sum(), df_awards.info(), df_awards.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4386961 entries, 0 to 4386960
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   player_id      int64 
 1   player_number  string
 2   type           string
 3   rank           string
dtypes: int64(1), string(3)
memory usage: 133.9 MB


player_id        0
player_number    0
type             0
rank             0
dtype: int64

None

,player_id,player_number,type,rank
0,3695653,8052557,section,1
1,3695652,tmp:1de5c0e8-a5a2-4433-a232-caa798c84939,section,1
2,3695646,tmp:1238a3ab-2652-4f75-921b-767aae646b2a,section,2
3,3695647,tmp:9f01395a-c179-4283-adf3-886ff6aed8aa,section,2
4,3695641,5637562,section,3/4


In [13]:
sections_dtypes = {
    'section_id':'int64',
    'session_id':'int64',
    'section_name':'string',
}
df_sections = dfs['sections']
df_sections = df_sections.astype(sections_dtypes)
display(df_sections.isna().sum(), df_sections.info(), df_sections.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341047 entries, 0 to 341046
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   section_id    341047 non-null  int64 
 1   session_id    341047 non-null  int64 
 2   section_name  341047 non-null  string
dtypes: int64(2), string(1)
memory usage: 7.8 MB


section_id      0
session_id      0
section_name    0
dtype: int64

None

,section_id,session_id,section_name
0,119248,100961,M
1,129563,110886,A
2,133527,114736,F
3,145352,126385,T
4,145382,126418,B


In [14]:
sessions_dtypes = {
    'session_id':'int64',
    'event_id':'int64',
    'hand_record_id':'string', # can contain SHUFFLE, etc.
    'game_date':'string'
}
df_sessions = dfs['sessions']
df_sessions = df_sessions.astype(sessions_dtypes)
display(df_sessions.isna().sum(), df_sessions.info(), df_sessions.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313223 entries, 0 to 313222
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   session_id      313223 non-null  int64 
 1   event_id        313223 non-null  int64 
 2   hand_record_id  260351 non-null  string
 3   game_date       313223 non-null  string
dtypes: int64(2), string(2)
memory usage: 9.6 MB


session_id            0
event_id              0
hand_record_id    52872
game_date             0
dtype: int64

None

,session_id,event_id,hand_record_id,game_date
0,100961,100961,77908,2020-02-17 00:00:00
1,110886,110886,SHUFFLE,2020-02-24 00:00:00
2,114736,114736,86634,2020-03-02 00:00:00
3,126385,126385,94064,2020-03-09 00:00:00
4,126418,126418,94082,2020-03-16 00:00:00


In [15]:
events_dtypes = {
    'event_id':'int64',
    'event_name':'string',
    'club_name':'string',
    'club_id_number':'int64',
    'event_type':'string',
    'rating':'int8',
    'board_scoring_method':'string',
    'stratification_type':'string',
    'winner_type':'int8',
    'club_class':'int8'
}
df_events = dfs['events']
df_events = df_events.astype(events_dtypes)
display(df_events.isna().sum(), df_events.info(), df_events.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313220 entries, 0 to 313219
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   event_id              313220 non-null  int64 
 1   event_name            313220 non-null  string
 2   club_name             313220 non-null  string
 3   club_id_number        313220 non-null  int64 
 4   event_type            313220 non-null  string
 5   rating                313220 non-null  int8  
 6   board_scoring_method  313220 non-null  string
 7   stratification_type   313220 non-null  string
 8   winner_type           313220 non-null  int8  
 9   club_class            313220 non-null  int8  
dtypes: int64(2), int8(3), string(5)
memory usage: 17.6 MB


event_id                0
event_name              0
club_name               0
club_id_number          0
event_type              0
rating                  0
board_scoring_method    0
stratification_type     0
winner_type             0
club_class              0
dtype: int64

None

,event_id,event_name,club_name,club_id_number,event_type,rating,board_scoring_method,stratification_type,winner_type,club_class
0,100961,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4
1,110886,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4
2,114736,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4
3,126385,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4
4,126418,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4


In [16]:
strats_dtypes = {
    'strat_id':'string',
    'event_id':'int64',
    'strats_number':'int8',
    'strats_label':'string',
    'strats_limit':'int16'
}
df_strats = dfs['strats']
df_strats = df_strats.astype(strats_dtypes)
display(df_strats.isna().sum(), df_strats.info(), df_strats.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874295 entries, 0 to 874294
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   strat_id       874295 non-null  string
 1   event_id       874295 non-null  int64 
 2   strats_number  874295 non-null  int8  
 3   strats_label   874295 non-null  string
 4   strats_limit   874295 non-null  int16 
dtypes: int16(1), int64(1), int8(1), string(2)
memory usage: 22.5 MB


strat_id         0
event_id         0
strats_number    0
strats_label     0
strats_limit     0
dtype: int64

None

,strat_id,event_id,strats_number,strats_label,strats_limit
0,274629,100961,1,A,0
1,301290,110886,1,A,0
2,311626,114736,1,A,0
3,342416,126385,1,A,0
4,342417,126385,2,B,100


In [17]:
strat_place_dtypes = {
    'strat_place_id':'int64',
    'pair_summary_id':'int64',
    'strat_place_number':'int8',
    'strat_place_rank':'string',
    'strat_place_type':'string'
}
df_strat_place = dfs['strat_place']
df_strat_place = df_strat_place.astype(strat_place_dtypes)
display(df_strat_place.isna().sum(), df_strat_place.info(), df_strat_place.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5458905 entries, 0 to 5458904
Data columns (total 5 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   strat_place_id      int64 
 1   pair_summary_id     int64 
 2   strat_place_number  int8  
 3   strat_place_rank    string
 4   strat_place_type    string
dtypes: int64(2), int8(1), string(2)
memory usage: 171.8 MB


strat_place_id        0
pair_summary_id       0
strat_place_number    0
strat_place_rank      0
strat_place_type      0
dtype: int64

None

,strat_place_id,pair_summary_id,strat_place_number,strat_place_rank,strat_place_type
0,1997656,1848542,1,1,Section
1,1997654,1848539,1,2,Section
2,1997653,1848536,1,3/4,Section
3,1997655,1848540,1,3/4,Section
4,2172350,2010256,1,1,Section


In [18]:
stratValues_dtypes = {
    'stratValues_id':'string',
    'event_id':'int64',
    'stratValues_label':'string',
    'stratValues_limit':'int32'
}
df_stratValues = dfs['stratValues']
df_stratValues = df_stratValues.astype(stratValues_dtypes)
display(df_stratValues.isna().sum(), df_stratValues.info(), df_stratValues.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874295 entries, 0 to 874294
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   stratValues_id     874295 non-null  string
 1   event_id           874295 non-null  int64 
 2   stratValues_label  874295 non-null  string
 3   stratValues_limit  874295 non-null  int32 
dtypes: int32(1), int64(1), string(2)
memory usage: 23.3 MB


stratValues_id       0
event_id             0
stratValues_label    0
stratValues_limit    0
dtype: int64

None

,stratValues_id,event_id,stratValues_label,stratValues_limit
0,100961-1,100961,A,0
1,110886-1,110886,A,0
2,114736-1,114736,A,0
3,126385-1,126385,A,0
4,126385-2,126385,B,100


In [19]:
# takes 8s
# create df from merging of pair_summaries and players dataframes on 'pair_summaries'.
df_pair_summaries_players = pd.merge(df_pair_summaries,df_players,on='pair_summary_id',how='inner')
df_pair_summaries_players.sort_values('pair_summary_id')

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible,player_id,player_number,player_name,mp_total
5353870,41,3,12,NS,1,48.52,<NA>,82,4446976,"Wetzel, Tutty",2358.31
5353869,41,3,12,NS,1,48.52,<NA>,81,1838512,"Yue, Raymond",1039.74
5353863,42,3,11,NS,1,52.59,<NA>,84,3264696,"Falk, Charlotte",7601.08
5353864,42,3,11,NS,1,52.59,<NA>,83,5674158,"Kellermeyer, Virginia(swap names)",3394.11
5353858,43,3,10,NS,1,57.78,<NA>,85,8594279,"Gupta, Satish(swap names)",2885.69
5353857,43,3,10,NS,1,57.78,<NA>,86,2351811,"Lucks, Sybil",2638.80
5353872,44,3,2,NS,1,42.59,<NA>,88,2728524,"Bahry, Sharon",3329.61
5353871,44,3,2,NS,1,42.59,<NA>,87,1302604,"Campbell, Patrick",18.37
5353867,45,3,9,NS,1,50.00,<NA>,90,2123991,"Scoggin, Richard",3834.03
5353868,45,3,9,NS,1,50.00,<NA>,89,4695429,"Nojima, Kazuko(swap names)",1581.76


In [20]:
# todo: drop NAs
len(df_pair_summaries_players),df_pair_summaries_players.isna().sum()

(10880585,
 pair_summary_id          0
 section_id               0
 pair_number              0
 direction           952742
 strat                   64
 percentage           15998
 is_eligible        1757478
 player_id                0
 player_number            0
 player_name              0
 mp_total            305921
 dtype: int64)

In [21]:
# takes 10s
# merge awards on player_id
# todo: assert each player_id has same player_number 
df_pair_summaries_players_awards = pd.merge(df_pair_summaries_players,df_awards,on=['player_id','player_number'],how='inner')
df_pair_summaries_players_awards.sort_values('pair_summary_id')

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible,player_id,player_number,player_name,mp_total,type,rank
2168669,42,3,11,NS,1,52.59,<NA>,83,5674158,"Kellermeyer, Virginia(swap names)",3394.11,section,5/6
2168668,42,3,11,NS,1,52.59,<NA>,84,3264696,"Falk, Charlotte",7601.08,section,5/6
2168663,43,3,10,NS,1,57.78,<NA>,85,8594279,"Gupta, Satish(swap names)",2885.69,section,2
2168662,43,3,10,NS,1,57.78,<NA>,86,2351811,"Lucks, Sybil",2638.80,section,2
2168660,47,3,1,NS,1,60.37,<NA>,94,5697522,"Martin, Betty",6194.71,section,1
2168661,47,3,1,NS,1,60.37,<NA>,93,2170833,"Robbins, Bob(swap names)",2978.84,section,1
2168671,48,3,4,NS,1,52.59,<NA>,96,5827191,"Lane, Kathy",2677.40,section,5/6
2168670,48,3,4,NS,1,52.59,<NA>,95,8644756,"Spalding, Lorita",2741.90,section,5/6
2168667,49,3,8,NS,2,57.04,<NA>,97,5549337,"Kornbluth, Bobbi(swap names)",1068.37,section,3/4
2168666,49,3,8,NS,2,57.04,<NA>,98,7892268,"Clarke, George",1047.93,section,3/4


In [22]:
# todo: drop NAs
len(df_pair_summaries_players_awards),df_pair_summaries_players_awards.isna().sum()

(4384989,
 pair_summary_id         0
 section_id              0
 pair_number             0
 direction          412840
 strat                   0
 percentage           5795
 is_eligible        872935
 player_id               0
 player_number           0
 player_name             0
 mp_total            60894
 type                    0
 rank                    0
 dtype: int64)

In [23]:
# takes 2s
df_pair_summaries_players_awards_sections = pd.merge(df_pair_summaries_players_awards,df_sections,on='section_id',how='inner')
df_pair_summaries_players_awards_sections.head()

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible,player_id,player_number,player_name,mp_total,type,rank,session_id,section_name
0,1848542,119248,4,<NA>,1,62.50,True,3695653,8052557,"Robson, Dave",693.89,section,1,100961,M
1,1848542,119248,4,<NA>,1,62.50,True,3695652,tmp:1de5c0e8-a5a2-4433-a232-caa798c84939,"Gamache, Len(swap names)",19.00,section,1,100961,M
2,1848539,119248,1,<NA>,1,57.29,True,3695646,tmp:1238a3ab-2652-4f75-921b-767aae646b2a,"Steele, Bruce",16.00,section,2,100961,M
3,1848539,119248,1,<NA>,1,57.29,True,3695647,tmp:9f01395a-c179-4283-adf3-886ff6aed8aa,"Hlady, Henny",NaN,section,2,100961,M
4,1848536,119248,3,<NA>,1,51.04,True,3695641,5637562,"Williams, Peter",139.21,section,3/4,100961,M


In [24]:
# todo: drop NAs
len(df_pair_summaries_players_awards_sections),df_pair_summaries_players_awards_sections.isna().sum()

(4384989,
 pair_summary_id         0
 section_id              0
 pair_number             0
 direction          412840
 strat                   0
 percentage           5795
 is_eligible        872935
 player_id               0
 player_number           0
 player_name             0
 mp_total            60894
 type                    0
 rank                    0
 session_id              0
 section_name            0
 dtype: int64)

In [25]:
# takes 2s
df_pair_summaries_players_awards_sections_sessions = pd.merge(df_pair_summaries_players_awards_sections,df_sessions,on='session_id',how='inner')
df_pair_summaries_players_awards_sections_sessions.head()

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible,player_id,player_number,player_name,mp_total,type,rank,session_id,section_name,event_id,hand_record_id,game_date
0,1848542,119248,4,<NA>,1,62.50,True,3695653,8052557,"Robson, Dave",693.89,section,1,100961,M,100961,77908,2020-02-17 00:00:00
1,1848542,119248,4,<NA>,1,62.50,True,3695652,tmp:1de5c0e8-a5a2-4433-a232-caa798c84939,"Gamache, Len(swap names)",19.00,section,1,100961,M,100961,77908,2020-02-17 00:00:00
2,1848539,119248,1,<NA>,1,57.29,True,3695646,tmp:1238a3ab-2652-4f75-921b-767aae646b2a,"Steele, Bruce",16.00,section,2,100961,M,100961,77908,2020-02-17 00:00:00
3,1848539,119248,1,<NA>,1,57.29,True,3695647,tmp:9f01395a-c179-4283-adf3-886ff6aed8aa,"Hlady, Henny",NaN,section,2,100961,M,100961,77908,2020-02-17 00:00:00
4,1848536,119248,3,<NA>,1,51.04,True,3695641,5637562,"Williams, Peter",139.21,section,3/4,100961,M,100961,77908,2020-02-17 00:00:00


In [26]:
# todo: drop NAs
len(df_pair_summaries_players_awards_sections_sessions),df_pair_summaries_players_awards_sections_sessions.isna().sum()

(4384989,
 pair_summary_id         0
 section_id              0
 pair_number             0
 direction          412840
 strat                   0
 percentage           5795
 is_eligible        872935
 player_id               0
 player_number           0
 player_name             0
 mp_total            60894
 type                    0
 rank                    0
 session_id              0
 section_name            0
 event_id                0
 hand_record_id      57365
 game_date               0
 dtype: int64)

In [27]:
# takes 3s
df_pair_summaries_players_awards_sections_sessions_events = pd.merge(df_pair_summaries_players_awards_sections_sessions,df_events,on='event_id',how='inner')
df_pair_summaries_players_awards_sections_sessions_events.head()

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible,player_id,player_number,player_name,mp_total,type,rank,session_id,section_name,event_id,hand_record_id,game_date,event_name,club_name,club_id_number,event_type,rating,board_scoring_method,stratification_type,winner_type,club_class
0,1848542,119248,4,<NA>,1,62.50,True,3695653,8052557,"Robson, Dave",693.89,section,1,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4
1,1848542,119248,4,<NA>,1,62.50,True,3695652,tmp:1de5c0e8-a5a2-4433-a232-caa798c84939,"Gamache, Len(swap names)",19.00,section,1,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4
2,1848539,119248,1,<NA>,1,57.29,True,3695646,tmp:1238a3ab-2652-4f75-921b-767aae646b2a,"Steele, Bruce",16.00,section,2,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4
3,1848539,119248,1,<NA>,1,57.29,True,3695647,tmp:9f01395a-c179-4283-adf3-886ff6aed8aa,"Hlady, Henny",NaN,section,2,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4
4,1848536,119248,3,<NA>,1,51.04,True,3695641,5637562,"Williams, Peter",139.21,section,3/4,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4


In [28]:
# todo: drop NAs
len(df_pair_summaries_players_awards_sections_sessions_events),df_pair_summaries_players_awards_sections_sessions_events.isna().sum()

(4384989,
 pair_summary_id              0
 section_id                   0
 pair_number                  0
 direction               412840
 strat                        0
 percentage                5795
 is_eligible             872935
 player_id                    0
 player_number                0
 player_name                  0
 mp_total                 60894
 type                         0
 rank                         0
 session_id                   0
 section_name                 0
 event_id                     0
 hand_record_id           57365
 game_date                    0
 event_name                   0
 club_name                    0
 club_id_number               0
 event_type                   0
 rating                       0
 board_scoring_method         0
 stratification_type          0
 winner_type                  0
 club_class                   0
 dtype: int64)

In [29]:
df_strats.sort_values('event_id').head(10)

,strat_id,event_id,strats_number,strats_label,strats_limit
453848,8,3,2,B,1500
453847,7,3,1,A,0
623399,11,5,1,A,0
454438,13,6,2,B,200
454437,12,6,1,A,1000
454526,16,7,3,C,500
454525,15,7,2,B,1500
454524,14,7,1,A,0
454612,18,8,2,B,1000
454611,17,8,1,A,0


In [30]:
# Takes 30s
# There can be multiple strats per event. Make a list of strats per event.
df_strats_agg = df_strats.groupby('event_id').agg(list)
df_strats_agg.head(10)

,strat_id,strats_number,strats_label,strats_limit
event_id,,,,
3,"[7, 8]","[1, 2]","[A, B]","[0, 1500]"
5,[11],[1],[A],[0]
6,"[12, 13]","[1, 2]","[A, B]","[1000, 200]"
7,"[14, 15, 16]","[1, 2, 3]","[A, B, C]","[0, 1500, 500]"
8,"[17, 18]","[1, 2]","[A, B]","[0, 1000]"
10,"[22, 23, 24]","[1, 2, 3]","[A, B, C]","[0, 1250, 300]"
11,"[25, 26, 27]","[1, 2, 3]","[A, B, C]","[200, 50, 20]"
12,"[28, 29, 30]","[1, 2, 3]","[A, B, C]","[0, 200, 100]"
13,"[31, 32, 33]","[1, 2, 3]","[A, B, C]","[0, 500, 100]"


In [31]:
# takes 3s
df_pair_summaries_players_awards_sections_sessions_events_strats = pd.merge(df_pair_summaries_players_awards_sections_sessions_events,df_strats_agg,on='event_id',how='inner')
df_pair_summaries_players_awards_sections_sessions_events_strats.head()

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible,player_id,player_number,player_name,mp_total,type,rank,session_id,section_name,event_id,hand_record_id,game_date,event_name,club_name,club_id_number,event_type,rating,board_scoring_method,stratification_type,winner_type,club_class,strat_id,strats_number,strats_label,strats_limit
0,1848542,119248,4,<NA>,1,62.50,True,3695653,8052557,"Robson, Dave",693.89,section,1,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0]
1,1848542,119248,4,<NA>,1,62.50,True,3695652,tmp:1de5c0e8-a5a2-4433-a232-caa798c84939,"Gamache, Len(swap names)",19.00,section,1,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0]
2,1848539,119248,1,<NA>,1,57.29,True,3695646,tmp:1238a3ab-2652-4f75-921b-767aae646b2a,"Steele, Bruce",16.00,section,2,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0]
3,1848539,119248,1,<NA>,1,57.29,True,3695647,tmp:9f01395a-c179-4283-adf3-886ff6aed8aa,"Hlady, Henny",NaN,section,2,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0]
4,1848536,119248,3,<NA>,1,51.04,True,3695641,5637562,"Williams, Peter",139.21,section,3/4,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0]


In [32]:
# todo: drop NAs
len(df_pair_summaries_players_awards_sections_sessions_events_strats),df_pair_summaries_players_awards_sections_sessions_events_strats.isna().sum()

(4384989,
 pair_summary_id              0
 section_id                   0
 pair_number                  0
 direction               412840
 strat                        0
 percentage                5795
 is_eligible             872935
 player_id                    0
 player_number                0
 player_name                  0
 mp_total                 60894
 type                         0
 rank                         0
 session_id                   0
 section_name                 0
 event_id                     0
 hand_record_id           57365
 game_date                    0
 event_name                   0
 club_name                    0
 club_id_number               0
 event_type                   0
 rating                       0
 board_scoring_method         0
 stratification_type          0
 winner_type                  0
 club_class                   0
 strat_id                     0
 strats_number                0
 strats_label                 0
 strats_limit                 

In [33]:
# Takes 7m!
# There can be multiple strat_places per pair_summary_id. Make a list of strat_places per pair_summary_id.
df_strat_place_agg = df_strat_place.groupby('pair_summary_id').agg(list)
df_strat_place_agg.head(10)

,strat_place_id,strat_place_number,strat_place_rank,strat_place_type
pair_summary_id,,,,
42,[48],[1],[5/6],[Section]
43,"[49, 50]","[1, 1]","[2, 2]","[Section, Event]"
47,"[51, 52]","[1, 1]","[1, 1]","[Section, Event]"
48,[53],[1],[5/6],[Section]
49,"[54, 55, 56]","[1, 1, 2]","[3/4, 3/4, 1]","[Section, Event, Section]"
50,"[57, 58]","[1, 1]","[3/4, 3/4]","[Section, Event]"
52,[59],[2],[2],[Section]
65,[66],[1],[2/4],[Section]
66,[67],[1],[2/4],[Section]


In [34]:
# takes 6s
df_pair_summaries_players_awards_sections_sessions_events_strats_strat_place = pd.merge(df_pair_summaries_players_awards_sections_sessions_events_strats,df_strat_place_agg,on=['pair_summary_id'],how='inner')
df_pair_summaries_players_awards_sections_sessions_events_strats_strat_place.head()

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible,player_id,player_number,player_name,mp_total,type,rank,session_id,section_name,event_id,hand_record_id,game_date,event_name,club_name,club_id_number,event_type,rating,board_scoring_method,stratification_type,winner_type,club_class,strat_id,strats_number,strats_label,strats_limit,strat_place_id,strat_place_number,strat_place_rank,strat_place_type
0,1848542,119248,4,<NA>,1,62.50,True,3695653,8052557,"Robson, Dave",693.89,section,1,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997656],[1],[1],[Section]
1,1848542,119248,4,<NA>,1,62.50,True,3695652,tmp:1de5c0e8-a5a2-4433-a232-caa798c84939,"Gamache, Len(swap names)",19.00,section,1,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997656],[1],[1],[Section]
2,1848539,119248,1,<NA>,1,57.29,True,3695646,tmp:1238a3ab-2652-4f75-921b-767aae646b2a,"Steele, Bruce",16.00,section,2,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997654],[1],[2],[Section]
3,1848539,119248,1,<NA>,1,57.29,True,3695647,tmp:9f01395a-c179-4283-adf3-886ff6aed8aa,"Hlady, Henny",NaN,section,2,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997654],[1],[2],[Section]
4,1848536,119248,3,<NA>,1,51.04,True,3695641,5637562,"Williams, Peter",139.21,section,3/4,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997653],[1],[3/4],[Section]


In [35]:
# todo: drop NAs
len(df_pair_summaries_players_awards_sections_sessions_events_strats_strat_place),df_pair_summaries_players_awards_sections_sessions_events_strats_strat_place.isna().sum()

(4331294,
 pair_summary_id              0
 section_id                   0
 pair_number                  0
 direction               406257
 strat                        0
 percentage                5605
 is_eligible             859021
 player_id                    0
 player_number                0
 player_name                  0
 mp_total                 59487
 type                         0
 rank                         0
 session_id                   0
 section_name                 0
 event_id                     0
 hand_record_id           57202
 game_date                    0
 event_name                   0
 club_name                    0
 club_id_number               0
 event_type                   0
 rating                       0
 board_scoring_method         0
 stratification_type          0
 winner_type                  0
 club_class                   0
 strat_id                     0
 strats_number                0
 strats_label                 0
 strats_limit                 

In [36]:
# Takes 25s
# There can be multiple stratValues per event. Make a list of stratValues per event.
df_stratValues_agg = df_stratValues.groupby('event_id').agg(list)
df_stratValues_agg.head(10)

,stratValues_id,stratValues_label,stratValues_limit
event_id,,,
3,"[3-1, 3-2]","[A, B]","[0, 1500]"
5,[5-1],[A],[0]
6,"[6-1, 6-2]","[A, B]","[1000, 200]"
7,"[7-1, 7-2, 7-3]","[A, B, C]","[0, 1500, 500]"
8,"[8-1, 8-2]","[A, B]","[0, 1000]"
10,"[10-1, 10-2, 10-3]","[A, B, C]","[0, 1250, 300]"
11,"[11-1, 11-2, 11-3]","[A, B, C]","[200, 50, 20]"
12,"[12-1, 12-2, 12-3]","[A, B, C]","[0, 200, 100]"
13,"[13-1, 13-2, 13-3]","[A, B, C]","[0, 500, 100]"


In [37]:
# takes 5s
df_pair_summaries_players_awards_sections_sessions_events_strats_strat_place_stratValues = pd.merge(df_pair_summaries_players_awards_sections_sessions_events_strats_strat_place,df_stratValues_agg,on='event_id',how='inner')
df_pair_summaries_players_awards_sections_sessions_events_strats_strat_place_stratValues.head()

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible,player_id,player_number,player_name,mp_total,type,rank,session_id,section_name,event_id,hand_record_id,game_date,event_name,club_name,club_id_number,event_type,rating,board_scoring_method,stratification_type,winner_type,club_class,strat_id,strats_number,strats_label,strats_limit,strat_place_id,strat_place_number,strat_place_rank,strat_place_type,stratValues_id,stratValues_label,stratValues_limit
0,1848542,119248,4,<NA>,1,62.50,True,3695653,8052557,"Robson, Dave",693.89,section,1,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997656],[1],[1],[Section],[100961-1],[A],[0]
1,1848542,119248,4,<NA>,1,62.50,True,3695652,tmp:1de5c0e8-a5a2-4433-a232-caa798c84939,"Gamache, Len(swap names)",19.00,section,1,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997656],[1],[1],[Section],[100961-1],[A],[0]
2,1848539,119248,1,<NA>,1,57.29,True,3695646,tmp:1238a3ab-2652-4f75-921b-767aae646b2a,"Steele, Bruce",16.00,section,2,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997654],[1],[2],[Section],[100961-1],[A],[0]
3,1848539,119248,1,<NA>,1,57.29,True,3695647,tmp:9f01395a-c179-4283-adf3-886ff6aed8aa,"Hlady, Henny",NaN,section,2,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997654],[1],[2],[Section],[100961-1],[A],[0]
4,1848536,119248,3,<NA>,1,51.04,True,3695641,5637562,"Williams, Peter",139.21,section,3/4,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997653],[1],[3/4],[Section],[100961-1],[A],[0]


In [38]:
# todo: drop NAs
len(df_pair_summaries_players_awards_sections_sessions_events_strats_strat_place_stratValues),df_pair_summaries_players_awards_sections_sessions_events_strats_strat_place_stratValues.isna().sum()

(4331294,
 pair_summary_id              0
 section_id                   0
 pair_number                  0
 direction               406257
 strat                        0
 percentage                5605
 is_eligible             859021
 player_id                    0
 player_number                0
 player_name                  0
 mp_total                 59487
 type                         0
 rank                         0
 session_id                   0
 section_name                 0
 event_id                     0
 hand_record_id           57202
 game_date                    0
 event_name                   0
 club_name                    0
 club_id_number               0
 event_type                   0
 rating                       0
 board_scoring_method         0
 stratification_type          0
 winner_type                  0
 club_class                   0
 strat_id                     0
 strats_number                0
 strats_label                 0
 strats_limit                 

In [102]:
allpr_file = acblPath.joinpath('allpr.pkl')
df_allpr = pd.read_pickle(allpr_file)
df_allpr['player_number'] = df_allpr['player_id'].map(lambda r: r[0] if len(r) else None).astype('string')
# 'Rank', 'PR', 'Games', 'Unit' errored if 'Int...' (why?) so made 'float32'
df_allpr_dtypes = {'Rank':'float32','PR':'float32','Games':'float32','Unit':'string','Name':'string','player_id':'string'}
df_allpr = df_allpr.astype(df_allpr_dtypes)
df_allpr.rename({'Rank':'bpr_rank','PR':'bpr_pr','Games':'bpr_games','Unit':'bpr_unit'},axis='columns',inplace=True)
# obsoleted by filter() - df_allpr.drop('player_id',axis='columns',inplace=True)
display(len(df_allpr),df_allpr.info(),df_allpr.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4855 entries, 0 to 4854
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bpr_rank       4855 non-null   float32
 1   bpr_pr         4855 non-null   float32
 2   bpr_games      4855 non-null   float32
 3   bpr_unit       4855 non-null   string 
 4   Name           4855 non-null   string 
 5   player_id      4855 non-null   string 
 6   player_number  4823 non-null   string 
dtypes: float32(3), string(4)
memory usage: 246.5 KB


4855

None

,bpr_rank,bpr_pr,bpr_games,bpr_unit,Name,player_id,player_number
0,1.00,72.17,137.00,108,"Grossack, Zachary",[2250896],2250896
1,2.00,71.19,60.00,174,"Wold, Eddie",['3443949'],3443949
2,3.00,71.15,86.00,373,"Shi, Sylvia",[5420105],5420105
3,4.00,71.03,164.00,128,"Cappelletti, Mike",['2601087'],2601087
4,5.00,71.02,201.00,538,"Itabashi, Mark",['6811434'],6811434


In [119]:
# takes 8s
# using filter() to remove unwanted columns: df_allpr.filter(regex='bpr_|player_number')
# merging using how='left', instead of 'inner', because bpr is 8% of total.
df = pd.merge(df_pair_summaries_players_awards_sections_sessions_events_strats_strat_place_stratValues,df_allpr.filter(regex='bpr_|player_number'),on='player_number',how='left')
bpr_cols = df.filter(regex='bpr_')
# todo: revisit. removing NA like this implies categorical? Needs astype('string') otherwise reverts to 'object'.
for cols in bpr_cols:
    df[cols] = df[cols].fillna('').astype('string')
display(len(df),df.info(),df.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4333555 entries, 0 to 4333554
Data columns (total 42 columns):
 #   Column                Dtype  
---  ------                -----  
 0   pair_summary_id       int64  
 1   section_id            int64  
 2   pair_number           int64  
 3   direction             string 
 4   strat                 Int8   
 5   percentage            float32
 6   is_eligible           boolean
 7   player_id             int64  
 8   player_number         string 
 9   player_name           string 
 10  mp_total              float32
 11  type                  string 
 12  rank                  string 
 13  session_id            int64  
 14  section_name          string 
 15  event_id              int64  
 16  hand_record_id        string 
 17  game_date             string 
 18  event_name            string 
 19  club_name             string 
 20  club_id_number        int64  
 21  event_type            string 
 22  rating                int8   
 23  board_s

4333555

None

,pair_summary_id,section_id,pair_number,direction,strat,percentage,is_eligible,player_id,player_number,player_name,mp_total,type,rank,session_id,section_name,event_id,hand_record_id,game_date,event_name,club_name,club_id_number,event_type,rating,board_scoring_method,stratification_type,winner_type,club_class,strat_id,strats_number,strats_label,strats_limit,strat_place_id,strat_place_number,strat_place_rank,strat_place_type,stratValues_id,stratValues_label,stratValues_limit,bpr_rank,bpr_pr,bpr_games,bpr_unit
0,1848542,119248,4,<NA>,1,62.50,True,3695653,8052557,"Robson, Dave",693.89,section,1,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997656],[1],[1],[Section],[100961-1],[A],[0],,,,
1,1848542,119248,4,<NA>,1,62.50,True,3695652,tmp:1de5c0e8-a5a2-4433-a232-caa798c84939,"Gamache, Len(swap names)",19.00,section,1,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997656],[1],[1],[Section],[100961-1],[A],[0],,,,
2,1848539,119248,1,<NA>,1,57.29,True,3695646,tmp:1238a3ab-2652-4f75-921b-767aae646b2a,"Steele, Bruce",16.00,section,2,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997654],[1],[2],[Section],[100961-1],[A],[0],,,,
3,1848539,119248,1,<NA>,1,57.29,True,3695647,tmp:9f01395a-c179-4283-adf3-886ff6aed8aa,"Hlady, Henny",NaN,section,2,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997654],[1],[2],[Section],[100961-1],[A],[0],,,,
4,1848536,119248,3,<NA>,1,51.04,True,3695641,5637562,"Williams, Peter",139.21,section,3/4,100961,M,100961,77908,2020-02-17 00:00:00,Monday Evening Pairs,Cariboo Bridge Club 1,100040,PAIRS,1,MATCH_POINTS,HIGHEST,1,4,[274629],[1],[A],[0],[1997653],[1],[3/4],[Section],[100961-1],[A],[0],,,,


### End of Merges

In [120]:
# takes 4s
len(df),df.isna().sum()

(4333555,
 pair_summary_id              0
 section_id                   0
 pair_number                  0
 direction               406375
 strat                        0
 percentage                5608
 is_eligible             859574
 player_id                    0
 player_number                0
 player_name                  0
 mp_total                 59488
 type                         0
 rank                         0
 session_id                   0
 section_name                 0
 event_id                     0
 hand_record_id           57227
 game_date                    0
 event_name                   0
 club_name                    0
 club_id_number               0
 event_type                   0
 rating                       0
 board_scoring_method         0
 stratification_type          0
 winner_type                  0
 club_class                   0
 strat_id                     0
 strats_number                0
 strats_label                 0
 strats_limit                 

In [121]:
df.value_counts('strat')

strat
1    1550633
2    1432725
3    1350197
dtype: int64

In [122]:
len(df),df.value_counts('rank'),df['rank'].isna().sum(),df['rank'].isna().sum()/len(df)

(4333555,
 rank
 2        1242104
 1        1041585
 3         905279
 4         537141
 5         180833
 2/3        94641
 6          80208
 3/4        73661
 1/2        56835
 4/5        41809
 7          16391
 5/6        15934
 8           8665
 6/7         7422
 2/4         5427
 3/5         4636
 9           4245
 1/3         2951
 4/6         2690
 10          2492
 7/8         1370
 11          1105
 5/7         1078
 8/9          769
 12           697
           ...   
 14/15         18
 7/10          16
 20            16
 9/11          16
 21            14
 8/11          14
 11/13         12
 2/6           10
 9/12           9
 5/9            8
 11/14          8
 23             6
 22             6
 19             6
 12/14          6
 31             4
 29             4
 28             4
 27             4
 26             4
 16/17          4
 25             4
 30             2
 24             2
 13/15          2
 Length: 72, dtype: int64,
 0,
 0.0)

In [123]:
df.value_counts('type')

type
section       3530144
ssoveralls     803411
dtype: int64

In [124]:
df.value_counts('is_eligible')

is_eligible
True     3462016
False      11965
dtype: int64

In [125]:
# takes 13s
pair_summaries_file = acblPath.joinpath('pair_summaries_cleaned.pkl')
with open(pair_summaries_file, 'wb') as f:
    pickle.dump(df, f)
print(f"Saved {pair_summaries_file}: len:{len(df)} size:{pair_summaries_file.stat().st_size}")

Saved e:\bridge\data\acbl\pair_summaries_cleaned.pkl: len:4333555 size:1165750331
